In [1]:
# Dependencies
import json
import requests
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
# Specify the URL
request_url = 'https://api.usa.gov/crime/fbi/sapi/api/data/hatecrime/states/tx/COUNT?API_KEY=qg50lz2p4PFyRNeEtptg63BHzgpf2tNB3gwnpJhM'

response = requests.get(request_url)
if response.status_code == 200:
    print('code 200 == woohoo!')

code 200 == woohoo!


In [5]:
crime_data = response.json()
print(json.dumps(crime_data, indent=4, sort_keys=True))

{
    "pagination": {
        "count": 29,
        "page": 0,
        "pages": 1,
        "per_page": 0
    },
    "results": [
        {
            "data_year": 2002,
            "incident_count": 343,
            "offense_count": 459
        },
        {
            "data_year": 1994,
            "incident_count": 365,
            "offense_count": 523
        },
        {
            "data_year": 2000,
            "incident_count": 287,
            "offense_count": 353
        },
        {
            "data_year": 2012,
            "incident_count": 175,
            "offense_count": 233
        },
        {
            "data_year": 2010,
            "incident_count": 174,
            "offense_count": 208
        },
        {
            "data_year": 2013,
            "incident_count": 142,
            "offense_count": 188
        },
        {
            "data_year": 2019,
            "incident_count": 224,
            "offense_count": 260
        },
        {
            "data_year